In [7]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma , FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

# 
memory = ConversationBufferMemory(return_messages=True)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


# 1. 파일 로드 (loader => UnstructuredFileLoader , spliter => CharacterTextSplitter)
loader = UnstructuredFileLoader("../files/document.txt")
splitter = CharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=100,
)
docs = loader.load_and_split(
    text_splitter=splitter,
)

# 2. Embedding + cache 작업 (vector를 매기는 작업 + local에 cache로 저장)
cache_dir = LocalFileStore("../cache/")
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# 3. vector store를 통해 prompt에 관련 되어 있는 부분을 찾습니다.
vector_store = FAISS.from_documents(docs, cached_embeddings)
# print(len(vector_store.similarity_search("Aaronson 은 유죄인가요?")))

# 4. vecctor store의 값을 통해 관련된 document를 가져와 propmpt에 참조용으로 전달합니다.(Retriever) 
# 4.5 memory에 대한 내용도 추가합니다.
retriever = vector_store.as_retriever()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "당신은 유용한 도우미입니다. 아래 제공된 문맥만을 사용하여 질문에 답하세요. 답을 모를 경우, 모른다고 말하세요. 답을 지어내지 마세요:\n\n{context}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}")
    ]
)

# 5. chain 작성 + invoke (체인 실행)
chat = ChatOpenAI(temperature=0.1)

chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
    "history": load_memory
} | prompt | chat

# chain 실행 함수, 메모리에 질문 답변을 저장해 둡니다.
def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    return result.content

Created a chunk of size 1444, which is longer than the specified 1200
Created a chunk of size 1251, which is longer than the specified 1200
Created a chunk of size 1493, which is longer than the specified 1200
Created a chunk of size 1458, which is longer than the specified 1200
Created a chunk of size 1411, which is longer than the specified 1200
Created a chunk of size 1417, which is longer than the specified 1200
Created a chunk of size 1345, which is longer than the specified 1200
Created a chunk of size 1339, which is longer than the specified 1200
Created a chunk of size 1288, which is longer than the specified 1200
Created a chunk of size 1444, which is longer than the specified 1200
Created a chunk of size 1496, which is longer than the specified 1200


In [2]:
questions = ["Aaronson 은 유죄인가요?", "그가 테이블에 어떤 메시지를 썼나요?", "Julia 는 누구인가요?"]
invoke_chain("Aaronson 은 유죄인가요?")

'네, Jones, Aaronson, 그리고 Rutherford은 그들이 기소된 범죄로 유죄입니다.'

In [3]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

'죄송합니다. 제가 제공받은 문맥에서는 Aaronson이 테이블에 어떤 메시지를 썼는지에 대한 정보가 없습니다.'

In [4]:
invoke_chain("Julia 는 누구인가요?")

'Julia는 주인공과 관련된 인물로, 그의 사랑이자 도움이 필요한 상황에 있는 사람입니다.'

In [5]:
invoke_chain("Aaronson 은 유죄인가요?")

'네, Jones, Aaronson, 그리고 Rutherford은 그들이 기소된 범죄로 유죄입니다.'

In [6]:
load_memory(_)

[HumanMessage(content='Aaronson 은 유죄인가요?'),
 AIMessage(content='네, Jones, Aaronson, 그리고 Rutherford은 그들이 기소된 범죄로 유죄입니다.'),
 HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'),
 AIMessage(content='죄송합니다. 제가 제공받은 문맥에서는 Aaronson이 테이블에 어떤 메시지를 썼는지에 대한 정보가 없습니다.'),
 HumanMessage(content='Julia 는 누구인가요?'),
 AIMessage(content='Julia는 주인공과 관련된 인물로, 그의 사랑이자 도움이 필요한 상황에 있는 사람입니다.'),
 HumanMessage(content='Aaronson 은 유죄인가요?'),
 AIMessage(content='네, Jones, Aaronson, 그리고 Rutherford은 그들이 기소된 범죄로 유죄입니다.')]